In [ ]:
import pandas as pd

In [ ]:
train = pd.read_pickle("train.pkl").dropna()
test = pd.read_pickle("test.pkl").dropna()

train.head()

,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,comment_tokens,...,parent_comment_hashtag_count,comment_number_count,parent_comment_number_count,comment_bow,parent_comment_bow,comment_tdidf,parent_comment_tdidf,comment_pos,parent_comment_pos,label
120111,they do not count because they make my argumen...,ZaeedGudamnMassani,casualiama,4,4,0,2015-12,2015-12-07 16:31:54,what is your argument about places like austra...,"[not, count, make, argument, worse]",...,0,0,1,"(0, 10813)\t1\n (0, 9560)\t1\n (0, 3677)\t...","(0, 14247)\t1\n (0, 22038)\t1\n (0, 14333)...","(0, 17468)\t0.526391431796563\n (0, 1135)\t...","(0, 2660)\t0.39323355007458427\n (0, 1990)\...","[(they, PRP), (do, VBP), (not, RB), (count, VB...","[(what, WP), (is, VBZ), (your, PRP$), (argumen...",1
162231,"it is the alt-firelink music, once you give he...",NobleSC3,darksouls3,1,-1,-1,2016-10,2016-10-24 22:27:45,secret betrayal? what is this? so i was doing ...,"[alt-firelink, music, ,, give, eye, firekeeper]",...,0,0,2,"(0, 6737)\t1\n (0, 5723)\t1\n (0, 10416)\t...","(0, 24261)\t1\n (0, 2500)\t1\n (0, 1799)\t...","(0, 837)\t0.6007913384547182\n (0, 10416)\t...","(0, 18208)\t0.148063849969385\n (0, 7816)\t...","[(it, PRP), (is, VBZ), (the, DT), (alt-firelin...","[(secret, JJ), (betrayal, NN), (?, .), (what, ...",0
89073,"so, um... where is the roller coaster?",erisawesome,Minecraft,1,1,0,2015-12,2015-12-22 03:42:20,the matterhorn in minecraft,"[,, um, ..., roller, coaster, ?]",...,0,0,0,"(0, 16443)\t1\n (0, 13496)\t1\n (0, 3106)\t1","(0, 15546)\t1","(0, 3106)\t0.6177210153841626\n (0, 13496)\...","(0, 15546)\t1.0","[(so, RB), (,, ,), (um, NN), (..., :), (where,...","[(the, DT), (matterhorn, NN), (in, IN), (minec...",1
104299,its the best super hero comic in the universe!,JadesterZ,ImageComics,3,3,0,2015-01,2015-01-04 08:25:58,i really hope more people start reading invinc...,"[best, super, hero, comic, universe, !]",...,0,0,0,"(0, 15339)\t1\n (0, 1765)\t1\n (0, 7392)\t...","(0, 11777)\t1\n (0, 17908)\t1\n (0, 19674)...","(0, 16576)\t0.5039666161296876\n (0, 3221)\...","(0, 12890)\t0.4795315811964264\n (0, 5188)\...","[(its, PRP$), (the, DT), (best, JJS), (super, ...","[(i, NNS), (really, RB), (hope, VBP), (more, J...",0
119666,the sad thing is i knew exactly what song this...,Sanguin3,tf2,1,1,0,2014-01,2014-01-27 10:30:34,moscow moscow hohohoho,"[sad, thing, knew, exactly, song, going, saw, .]",...,0,0,0,"(0, 15844)\t1\n (0, 6814)\t1\n (0, 5576)\t...","(0, 15936)\t2","(0, 14702)\t0.42382056409678726\n (0, 13788...","(0, 15936)\t1.0","[(the, DT), (sad, JJ), (thing, NN), (is, VBZ),...","[(moscow, NN), (moscow, NN), (hohohoho, NN)]",0


In [ ]:
train['label'].value_counts(normalize = True)
test['label'].value_counts(normalize = True)

In [ ]:
vocab = []
for comment in train['comment_tokens']:
   for word in comment:
      vocab.append(word)

vocab = list(set(vocab))
len(vocab)
n_vocab = len(vocab)

In [ ]:
words_per_comment = {unique_word: [0] * len(train['comment_tokens']) for unique_word in vocab}

for i, comment in enumerate(train['comment_tokens']):
   for word in comment:
      words_per_comment[word][i] += 1

word_counts = pd.DataFrame(words_per_comment)
word_counts.head()

In [ ]:
train_clean = pd.concat([train, word_counts], axis=1)
train_clean.head()

In [ ]:
# Isolating sarcastic and non-sarcastic messages first
sarcastic = train_clean[train_clean['label'] == 1]
non_sarcastic = train_clean[train_clean['label'] == 0]

# P(Sar), P(Non-sar), N(Sar), N(Non-sar)
p_sar = len(sarcastic) / len(train_clean)
p_non_sar = len(non_sarcastic) / len(train_clean)

n_words_per_sar_comment = sarcastic['comment_tokens'].apply(len)
n_sar = n_words_per_sar_comment.sum()

n_words_per_non_comment = non_sarcastic['comment_tokens'].apply(len)
n_non_sar = n_words_per_non_comment.sum()

alpha = 1

param_sar = {unique_word:0 for unique_word in vocab}
param_non_sar = {unique_word:0 for unique_word in vocab}

# Calculate parameters
for word in vocab:
   n_word_given_sar = sarcastic[word].sum()
   p_word_given_sar = (n_word_given_sar + alpha) / (n_sar + alpha*n_vocab)
   param_sar[word] = p_word_given_sar

   n_word_given_non_sar = non_sarcastic[word].sum()
   p_word_given_non_sar = (n_word_given_non_sar + alpha) / (n_non_sar + alpha*n_vocab)
   param_non_sar[word] = p_word_given_non_sar

In [ ]:
def naive_bayes(text):

   p_sar_given_text = p_sar
   p_non_sar_given_text = p_non_sar

   for word in text:
      if word in param_sar:
         p_sar_given_text *= param_sar[word]

      if word in param_non_sar:
         p_ham_given_message *= param_non_sar[word]

   if p_non_sar_given_text > p_sar_given_text:
      print('Label: 0')
   elif p_non_sar_given_text < p_sar_given_text:
      print('Label: 1')
   else:
      print('-')

In [ ]:
def testing_naive_bayes(text):

   p_sar_given_text = p_sar
   p_non_sar_given_text = p_non_sar

   for word in text:
      if word in param_sar:
         p_sar_given_text *= param_sar[word]

      if word in param_non_sar:
         p_non_sar_given_text *= param_non_sar[word]

   if p_non_sar_given_text > p_sar_given_text:
      return '0'
   elif p_non_sar_given_text < p_sar_given_text:
      return '1'
   else:
      return 'cannot tell'

In [ ]:
test['predicted_label'] = test['comment_tokens'].apply(testing_naive_bayes)
test.head()

In [ ]:
correct = 0
total = test.shape[0]

for row in test.iterrows():
   row = row[1]
   if row['label'] == row['predicted_label']:
      correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)